This is the work of Lydia Armini and Beatriz Cabrero-Daniel

Util functions

In [ ]:
def normalise_csp(csp):
    return csp

def csp_transformation1(csp):
    return csp

def csp_transformation2(csp):
    return csp

def csp_transformation3(csp):
    return csp

Load all datapoints (synthetic for now), change the format as needed (separate file or new columns)

Define metrics for (dis)similarity (try to keep it simple)

In [ ]:
def euclidean_distance(csp1, csp2):
    difference = 0
    return difference

def dynamictimewarping_distance(csp1, csp2):
    csp1 = normalise_csp(csp1, -1, 1)
    csp2 = normalise_csp(csp2, -1, 1)
    difference = 0
    return difference

def matrixprofilebased_distance(csp1, csp2):
    difference = 0
    return difference

For all pairs of datapoints, compute distance using all metrics

Export as Latex tables or heatmaps or both